In [1]:

# 1) Import necessary libraries
import pandas as pd
import re

# 2) Define File Paths
file_path_text = 'NoisyText.txt'        # raw noisy text file
file_path_csv  = 'vdoLinks.csv'         # CSV containing at least: youtubeId, title
output_path    = 'Cleaned_Comments_Output.txt'

# 3) Load Data
movie_list = pd.read_csv(file_path_csv)

with open(file_path_text, 'r', encoding='latin-1') as f:
    raw_data = f.read()

# 4) Calculate Initial Noise Metrics
letters_before = len(re.findall(r'[A-Za-z]', raw_data))

# 5) Define Slang and Replacement Logic
slang_dict = {
    "lol":  "(**)", "omg":  "(**)", "wtf": "(**)",
    "idk": "(**)", "btw": "(**)", "lmao": "(**)"
}

# Fix: remove stray parenthesis and escape keys + whole-word, case-insensitive
def replace_slang(text: str) -> str:
    for slang, replacement in slang_dict.items():
        pattern = re.compile(rf"\b{re.escape(slang)}\b", flags=re.IGNORECASE)
        text = pattern.sub(replacement, text)
    return text

# 6) Core Parsing and Cleaning Function
def parse_data(data: str) -> pd.DataFrame:
    sections = data.split('NewMovieDrPQRd')
    parsed = []

    for section in sections:
        lines = section.strip().split('\n')

        # Skip empty sections
        if not lines or len(lines[0].strip()) == 0:
            continue

        # First line is the YouTube ID
        movie_id = lines[0].strip()
        comments = []

        for line in lines[1:]:
            line = line.strip()

            # Skip empty lines or repeated ID
            if not line or line == movie_id:
                continue

            # Skip noisy/system lines (support both &lt; and literal <)
            if line.startswith('&lt;') or line.startswith('<') or line.startswith("'"):
                continue

            if any(skip in line.lower() for skip in [
                'http error', 'charmap', 'pt1m', '128238',
                'video has disabled comments'
            ]):
                continue

            # Replace slang
            line = replace_slang(line)

            # Keep the cleaned line
            comments.append(line)

        if not comments:
            comments = ['No comments were found']

        parsed.append({'youtubeId': movie_id, 'comments': comments})

    return pd.DataFrame(parsed)

# 7) Execute the Cleaning
parsed_df = parse_data(raw_data)

# 8) Merge the cleaned comments with movie list (ensure youtubeId as string on both)
parsed_df['youtubeId'] = parsed_df['youtubeId'].astype(str).str.strip()
if 'youtubeId' not in movie_list.columns:
    raise KeyError("CSV is missing required column: 'youtubeId'")
movie_list['youtubeId'] = movie_list['youtubeId'].astype(str).str.strip()

merged_df = pd.merge(parsed_df, movie_list, on='youtubeId', how='left')

# 9) Write the Output (match EXACT phrasing/casing from your example)
with open(output_path, 'w', encoding='utf-8') as f:
    for _, row in merged_df.iterrows():
        movie_title = row['title'] if ('title' in row and pd.notna(row['title'])) else row['youtubeId']
        f.write(f"Movie Name: {movie_title}\n")

        if row['comments'] == ['No comments were found']:
            f.write("No comments were found\n\n")  # keep blank line after
        else:
            f.write("The Comments are:\n")  # Capital C to match your sample
            for comment in row['comments']:
                f.write(comment + '\n')
            f.write('\n')  # blank line between movie sections

# 10) Calculate and Display Cleaning Efficiency
with open(output_path, 'r', encoding='utf-8') as f:
    cleaned_text = f.read()

letters_after = len(re.findall(r'[A-Za-z]', cleaned_text))
ratio = (letters_after / letters_before) if letters_before else 0.0

print('Letters Before:', letters_before)
print('Letters After :', letters_after)
print('Cleaning Ratio:', round(ratio, 3))


Letters Before: 11552036
Letters After : 3508525
Cleaning Ratio: 0.304
